In [1]:
from lxml import etree, html
from lxml.html.clean import clean_html
import pandas as pd
from datetime import date, datetime
import requests
import re
import os
import shutil
from ASECORP_BBDD import tagea_BBDD_ASECORP, devuelve_patrones

In [2]:
today = date.today()

In [3]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

In [4]:
# Define expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
#pattern = ['Ley [0-9]+\/[0-9]+','Ley Orgánica [0-9]+\/[0-9]+','Decreto [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden [0-9]+\/[0-9]+','Orden de [0-9]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento [0-9]+\/[0-9]+', 'Directiva [0-9]+\/[0-9]+','Directiva \(UE\) [0-9]+\/[0-9]+','Reglamento n\º [0-9]+\/[0-9]+', 'Reglamento \(UE\) [0-9]+\/[0-9]+','Reglamento \(UE\) n\º [0-9]+\/[0-9]+','Reglamento \(CE\) n\º [0-9]+\/[0-9]+','Reglamento \(CEE\) n\º [0-9]+\/[0-9]+','Reglamento de Ejecución \(UE\) [0-9]+\/[0-9]+','Reglamento de Ejecución \(UE\) n\º [0-9]+\/[0-9]+' ,'Reglamento de Ejecución [0-9]+\/[0-9]+','Reglamento Delegado [0-9]+\/[0-9]+','Reglamento Delegado \(UE\) [0-9]+\/[0-9]+','Reglamento Delegado \(UE\) n\º [0-9]+\/[0-9]+','Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+','Resolución de [0-9]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Acuerdo Multilateral RID [0-9]+\/[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión [0-9]+\/[0-9]+','Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+', 'Instrucción IS\-[0-9]+']
pattern = devuelve_patrones()

## Importa BBDD ASECORP

In [5]:
ASECORP_BBDD = pd.read_csv('./ASECORP/ExportNormes_20210126.csv', delimiter=';')

In [6]:
ASECORP_BBDD

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
0,RES SLT/2313/2020 CAT,2313,"Resolución SLT/2313/2020, de 24 de septiembre,...",CAT,Cataluña,NaN,Prevención,[2]
1,(zzz) RES 12/4/2007 AST,12/4/2007,"Resolución de 12 abril de 2007, de la Consejer...",AST,Asturias,NaN,M. Ambiente,[1]
2,A CORUÑA 10/7/2015,10/7/2015,Modificación de la Ordenanza de vertidos y ser...,GAL,Galicia,A Coruña,M. Ambiente,[1]
3,A CORUÑA 11/8/2014 (1),11/8/2014,Ordenanza municipal reguladora de los procedim...,GAL,Galicia,A Coruña,M. Ambiente,[1]
4,A CORUÑA 11/8/2014 (2),11/8/2014,Ordenanza de protección contra la contaminació...,GAL,Galicia,A Coruña,M. Ambiente,[1]
...,...,...,...,...,...,...,...,...
19152,ZUMAIA 1/3/2001,1/3/2001,Ordenanza reguladora de la gestión del ciclo i...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19153,ZUMAIA 12/11/1999,12/11/1999,Normativa complementaria de las Normas Subsidi...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19154,ZUMAIA 15/6/2012,15/6/2012,Modificación de la Ordenanza municipal de la g...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19155,ZUMAIA 6/2/2004,6/2/2004,Modificación de la Normativa complementaria de...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]


In [7]:
#ASECORP_BBDD_BOE = ASECORP_BBDD.loc[ASECORP_BBDD['Ambito'] == 'Europa']

In [8]:
ASECORP_BBDD_BOE = ASECORP_BBDD.loc[ASECORP_BBDD['Ambito'].isin (['España','Europa', 'Castilla la Mancha'])]

In [9]:
ASECORP_BBDD_BOE

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
23,ACU 1/2020 ESP,1,Acuerdo Multilateral RID 1/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
35,ACU 11/6/2007 ESP,11/6/2007,Entrada en vigor del Acuerdo entre el Reino de...,ESP,España,NaN,M. Ambiente,[1]
57,ACU 17/10/2006 ESP,17/10/2006,Aplicación provisional del Acuerdo entre el Re...,ESP,España,NaN,M. Ambiente,[1]
71,ACU 19/11/2018 ESP,19/11/2018,Acuerdo de la Junta de Gobierno de la Confeder...,ESP,España,NaN,M. Ambiente,[1]
82,ACU 2/2020 ESP,2,Acuerdo Multilateral RID 2/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
...,...,...,...,...,...,...,...,...
18646,TOMELLOSO 24/4/2019,24/4/2019,Modificación de la Ordenanza General de Protec...,CMA,Castilla la Mancha,Tomelloso,M. Ambiente,[1]
18647,TOMELLOSO 9/3/2011,9/3/2011,Ordenanza general de protección del medio ambi...,CMA,Castilla la Mancha,Tomelloso,M. Ambiente,[1]
18879,verificación de calidad,1,Norma verificación calidad alertas en produccion,ESP,España,-,Reg. Industrial,[3]
19118,YUNCOS 27/4/2010,27/4/2010,Ordenanza de convivencia ciudadana de Yuncos.,CMA,Castilla la Mancha,Yuncos,M. Ambiente,[1]


In [10]:
# Crea nueva columna vacía de tipo lista en ASECORP_BBDD
ASECORP_BBDD_BOE['Tags'] = [[] for i in range(len(ASECORP_BBDD_BOE))]

In [11]:
titulo = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    titulo.append(str(row['Titulo']) + str(re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)))
    ASECORP_BBDD_BOE['Tags'][i] = re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)

In [12]:
ASECORP_BBDD_BOE[1200:1250]

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma,Tags
4557,DIR 2003/34 EUR,34,Directiva 2003/34/CE del Parlamente Europeo y ...,EUR,Europa,NaN,M. Ambiente / Prevención,[1][2],"[Directiva 2003/34, Directiva 76/769]"
4558,DIR 2003/35 EUR,35,Directiva 2003/35/CE del Parlamento Europeo y ...,EUR,Europa,NaN,M. Ambiente,[1],[Directiva 2003/35]
4559,DIR 2003/36 EUR,36,Directiva 2003/36/CE del Parlamento Europeo y ...,EUR,Europa,NaN,M. Ambiente / Prevención,[1][2],"[Directiva 2003/36, Directiva 76/769]"
4560,DIR 2003/4 EUR,4,Directiva 2003/4/CE del Parlamento Europeo y d...,EUR,Europa,NaN,M. Ambiente,[1],"[Directiva 2003/4, Directiva 90/313]"
4561,DIR 2003/40 EUR,40,Directiva 2003/40/CE de la Comisión de 16 de m...,EUR,Europa,NaN,Seg. Alimentaria,[7],[Directiva 2003/40]
4562,DIR 2003/53 EUR,53,Directiva 2003/53/CE del Parlamento Europeo y ...,EUR,Europa,NaN,M. Ambiente / Prevención,[1][2],"[Directiva 2003/53, Directiva 76/769]"
4563,DIR 2003/58 EUR,58,Directiva 2003/58/CE del Parlamento Europeo y ...,EUR,Europa,NaN,M. Ambiente / Prevención,[1][2],"[Directiva 2003/58, Directiva 68/151]"
4564,DIR 2003/73 EUR,73,"Directiva 2003/73/CE de la Comisión, de 24 de ...",EUR,Europa,NaN,M. Ambiente,[1],"[Directiva 2003/73, Directiva 1999/94]"
4565,DIR 2003/76 EUR,76,Directiva 2003/76/CE de la comisión de 11 de a...,EUR,Europa,NaN,M. Ambiente,[1],"[Directiva 2003/76, Directiva 70/220]"
4566,DIR 2003/87 EUR,87,Directiva 2003/87/CE del parlamento europeo y ...,EUR,Europa,NaN,M. Ambiente,[1],"[Directiva 2003/87, Directiva 96/61]"


In [13]:
# titulo

In [14]:
len(titulo)

5423

In [15]:
## Busca patrones regex definidos en columna títulos del DF 
boletin = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    boletin.append(str(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)))

In [16]:
boletin[0:25]

["['Acuerdo Multilateral RID 1/2020']",
 '[]',
 '[]',
 '[]',
 "['Acuerdo Multilateral RID 2/2020']",
 '[]',
 "['Acuerdo Multilateral RID 1/2013']",
 '[]',
 '[]',
 '[]',
 '[]',
 "['Acuerdo Multilateral RID 3/2007']",
 "['Acuerdo Multilateral RID 4/2009']",
 '[]',
 '[]',
 "['Acuerdo Multilateral M-150']",
 "['Acuerdo Multilateral M-159']",
 "['Acuerdo multilateral M-168']",
 "['Acuerdo Multilateral M-170']",
 "['Acuerdo Multilateral M-171']",
 "['Acuerdo Multilateral M-173']",
 "['Acuerdo Multilateral M-175']",
 '[]',
 "['Acuerdo Multilateral M-178']",
 "['Acuerdo Multilateral M-198']"]

In [17]:
## Para cada fila de la BBDD recoge la expresión REGEX encontrada y si no existe no la incluye 
## en la lista resultante llamada 'boletin'. Además si no detecta expresión la cuenta como vacía
## en variable 'n_vacios', y la añade a la lista 'vacios' para inspeccionar posteriormente 
boletin = []
vacios = []
n_vacios = 0
for i, row in ASECORP_BBDD_BOE.iterrows():
    regex_result = re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)
    if len(regex_result) != 0:
        boletin.append(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE))
    else:
        n_vacios += 1
        vacios.append(row['Titulo'])

In [18]:
n_vacios

805

In [19]:
vacios[0:25]

['Entrada en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Aplicación provisional del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Acuerdo de la Junta de Gobierno de la Confederación Hidrográfica del Segura celebrada el día 30 de octubre de 2018 sobre aplicación de indisponibilidad o reducción de dotaciones de acuerdo al Plan Especial de Sequía y al Real Decreto que la regula',
 'Entrada en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre el traslado de residuos, hecho en Madrid el 29 de noviembre de 2011.',
 'Canje de Notas por el que se procede a la prórroga del mantenimiento en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 27 de enero de 2000.',
 'Aplicación provisional del

In [20]:
boletin[0:25]

[['Acuerdo Multilateral RID 1/2020'],
 ['Acuerdo Multilateral RID 2/2020'],
 ['Acuerdo Multilateral RID 1/2013'],
 ['Acuerdo Multilateral RID 3/2007'],
 ['Acuerdo Multilateral RID 4/2009'],
 ['Acuerdo Multilateral M-150'],
 ['Acuerdo Multilateral M-159'],
 ['Acuerdo multilateral M-168'],
 ['Acuerdo Multilateral M-170'],
 ['Acuerdo Multilateral M-171'],
 ['Acuerdo Multilateral M-173'],
 ['Acuerdo Multilateral M-175'],
 ['Acuerdo Multilateral M-178'],
 ['Acuerdo Multilateral M-198'],
 ['Acuerdo Multilateral M-214'],
 ['Acuerdo Multilateral M-218'],
 ['Acuerdo Multilateral M-226'],
 ['Acuerdo Multilateral M-228'],
 ['Acuerdo Multilateral M-231'],
 ['Acuerdo Multilateral M-236'],
 ['Acuerdo Multilateral M-245'],
 ['Acuerdo Multilateral M-254'],
 ['Acuerdo Multilateral M-259'],
 ['Acuerdo Multilateral M-265'],
 ['Acuerdo Multilateral M-268']]

In [21]:
# Flatten list of lists
boletin_ASECORP_flat_list = [item for sublist in boletin for item in sublist]

In [22]:
boletin_ASECORP_flat_list[0:25]

['Acuerdo Multilateral RID 1/2020',
 'Acuerdo Multilateral RID 2/2020',
 'Acuerdo Multilateral RID 1/2013',
 'Acuerdo Multilateral RID 3/2007',
 'Acuerdo Multilateral RID 4/2009',
 'Acuerdo Multilateral M-150',
 'Acuerdo Multilateral M-159',
 'Acuerdo multilateral M-168',
 'Acuerdo Multilateral M-170',
 'Acuerdo Multilateral M-171',
 'Acuerdo Multilateral M-173',
 'Acuerdo Multilateral M-175',
 'Acuerdo Multilateral M-178',
 'Acuerdo Multilateral M-198',
 'Acuerdo Multilateral M-214',
 'Acuerdo Multilateral M-218',
 'Acuerdo Multilateral M-226',
 'Acuerdo Multilateral M-228',
 'Acuerdo Multilateral M-231',
 'Acuerdo Multilateral M-236',
 'Acuerdo Multilateral M-245',
 'Acuerdo Multilateral M-254',
 'Acuerdo Multilateral M-259',
 'Acuerdo Multilateral M-265',
 'Acuerdo Multilateral M-268']

In [23]:
## Elimina duplicados
boletin_ASECORP_flat_list = list(set(boletin_ASECORP_flat_list))

In [24]:
boletin_ASECORP_flat_list[0:25]

['Orden Circular 30/2012',
 'Ley 24/2005',
 'Real Decreto 202/2006',
 'Real decreto 2210/1995',
 'Real Decreto 553/2020',
 'Reglamento (UE) 2019/1782',
 'Reglamento Delegado (UE) 2018/674',
 'Orden ITC/1857/2008',
 'Reglamento (UE) 2016/1005',
 'Orden ARM/1783/2011',
 'Directiva 2011/88',
 'Real Decreto 731/2020',
 'Reglamento 852/2004',
 'Directiva 2010/65',
 'Reglamento 1354/2007',
 'Directiva 2000/60',
 'Directiva 96/60',
 'Directiva 2003/3',
 'Nota de servicio 4/2001',
 'Orden de 20/11/2013',
 'Real Decreto 596/2015',
 'Real Decreto 365/2005',
 'Resolución de 7/8/1998',
 'Circular 1/2016',
 'Ley Orgánica 9/2013']

# Genera Fichero EXCEL de resultados

In [25]:
## Crea función que convierte lista a string en todas las columnas de tabla_analisis
## para evitar en presentación final los caracteres [' '] propios de las listas
def list2Str(lst):
    if type(lst) is list: # apply conversion to list columns
        return", ".join(lst)
    else:
        return lst

In [26]:
## Aplica función de conversión de listas a strings
ASECORP_BBDD_BOE_final = ASECORP_BBDD_BOE.apply(lambda x: [list2Str(i) for i in x])

In [27]:
ASECORP_BBDD_BOE_final.to_csv("./ASECORP/ASECORP_BBDD_Tageada_" + today.strftime("%Y%m%d") + ".csv", index=False) 